In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

In [149]:
df = pd.read_csv("data/cleaned_data.csv")

In [150]:
df.head()


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [ ]:
X = df.drop(columns=["average","total_score", "math_score"],axis=1)
Y = df["math_score"]

X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [152]:
Y.head()

0    72.666667
1    82.333333
2    92.666667
3    49.333333
4    76.333333
Name: average, dtype: float64

In [153]:
num_featues = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(exclude="int").columns

In [154]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer([
    ("OneHotEncoder", oh_transformer, cat_features),
    ("StandardScalar", numeric_transformer, num_featues),
])

In [155]:
X = preprocessor.fit_transform(X)

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 20), (200, 20))

In [157]:
def evalute_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [158]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "KNeighbors Regressor" : KNeighborsRegressor(),
    "Decesion Tree": DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoostRegressor" : CatBoostRegressor(),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

models_list = []
r2_List = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)

    model_train_mae,model_train_mse,model_train_rmae,model_train_r2 = evalute_model(Y_train, Y_train_pred)    
    model_test_mae,model_test_mse,model_test_rmae,model_test_r2 = evalute_model(Y_test, Y_test_pred)  

    print(list(models.keys())[i])
    models_list.append(list(models.keys())[i])

    print("Model Performace for Traing set")
    print("Root Mean Squared Error: {:.4f}".format(model_train_rmae))
    print("Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("R2 Score: {:.4f}".format(model_train_r2))
    
    print("Model Performace for Test set")
    print("Root Mean Squared Error: {:.4f}".format(model_test_rmae))
    print("Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("R2 Score: {:.4f}".format(model_test_r2))
    
    r2_List.append(model_test_r2)

    print("=" * 35)
    print("\n")

Linear Regression
Model Performace for Traing set
Root Mean Squared Error: 0.0000
Mean Absolute Error: 0.0000
R2 Score: 1.0000
Model Performace for Test set
Root Mean Squared Error: 0.0000
Mean Absolute Error: 0.0000
R2 Score: 1.0000


Lasso
Model Performace for Traing set
Root Mean Squared Error: 1.0645
Mean Absolute Error: 0.8506
R2 Score: 0.9943
Model Performace for Test set
Root Mean Squared Error: 1.1142
Mean Absolute Error: 0.8769
R2 Score: 0.9942


Ridge
Model Performace for Traing set
Root Mean Squared Error: 0.0080
Mean Absolute Error: 0.0065
R2 Score: 1.0000
Model Performace for Test set
Root Mean Squared Error: 0.0088
Mean Absolute Error: 0.0067
R2 Score: 1.0000


KNeighbors Regressor
Model Performace for Traing set
Root Mean Squared Error: 2.2573
Mean Absolute Error: 1.7680
R2 Score: 0.9745
Model Performace for Test set
Root Mean Squared Error: 3.0706
Mean Absolute Error: 2.2833
R2 Score: 0.9560


Decesion Tree
Model Performace for Traing set
Root Mean Squared Error: 0.0000

In [159]:

pd.DataFrame(list(zip(models_list, r2_List)), columns=["Model Name", "R2_score"]).sort_values(by=["R2_score"],ascending=False)

,Model Name,R2_score
0,Linear Regression,1.000000
2,Ridge,1.000000
6,XGBRegressor,0.994748
1,Lasso,0.994209
5,Random Forest Regressor,0.993504
7,CatBoostRegressor,0.993447
4,Decesion Tree,0.988231
8,AdaBoost Regressor,0.976803
3,KNeighbors Regressor,0.956016


# Liner Regression Model selected as per the evaluation. 

In [160]:
Lin_model = LinearRegression()
Lin_model = Lin_model.fit(X_train,Y_train)
y_pred = Lin_model.predict(X_test)
score = r2_score(Y_test,y_pred)*100
print("Accuracy Score for the model is %.2f" %score)

Accuracy Score for the model is 100.00
